<a href="https://colab.research.google.com/github/Dawoon-Jeong0523/Product-Science/blob/main/WebCrawler_GSMArena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config

In [ ]:
data_directory=r'/content/drive/MyDrive/Research/Product Science/Data'
output_directory=r'/content/drive/MyDrive/Research/Product Science/Output'

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
import time
import json
from collections import OrderedDict
import re
import requests

In [ ]:
!pip install selenium bs4 webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

options = ChromeOptions()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
options.add_argument('user-agent=' + user_agent)
options.add_argument("lang=ko_KR")
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("--no-sandbox")

# chrome driver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
import requests

url="https://www.gsmarena.com/"

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}
res=requests.get(url,headers=headers)
print(res.text)

# URL path

In [ ]:
import time
import random
from tqdm import tqdm

time.sleep(random.randint(1,10))

In [ ]:
import time
import random

input_URL="https://www.gsmarena.com/makers.php3"
product_url_list=[]

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}
session = requests.Session()
html = session.get(input_URL, headers=headers).content
soup = bs(html, 'html.parser')

temp_firm_url_list=[]
for item in soup.find("div",class_="st-text").find_all("a"):
  temp_firm_url_list.append("https://www.gsmarena.com/"+item["href"])


for temp_firm in tqdm(temp_firm_url_list):
  time.sleep(random.randint(1,10))
  html = session.get(temp_firm, headers=headers).content
  soup = bs(html, 'html.parser')

  # 특정기업(1개)의 page url 리스트
  one_firm_url_list=[]
  one_firm_url_list.append(temp_firm)
  try:
    for item in soup.find("div",class_="nav-pages").find_all("a"):
      one_firm_url_list.append("https://www.gsmarena.com/"+item["href"])
  except:
    pass

  for temp_url in one_firm_url_list:
    time.sleep(random.randint(1,10))
    html = session.get(temp_url, headers=headers).content
    soup = bs(html, 'html.parser')
    try:
      for item in soup.find("div",class_="makers").find_all("li"):
        product_url_list.append("https://www.gsmarena.com/"+item.find("a")["href"])
    except:
      print(temp_url)

In [ ]:
len(product_url_list)

In [ ]:
import pandas as pd

In [ ]:
temp_df=pd.DataFrame()
temp_df["product url"]=product_url_list
temp_df.to_csv(data_directory+"/product_url_list_2025.csv")

# Crawling 1-page

In [ ]:
import pandas as pd
product_url_list=pd.read_csv(data_directory+"/product_url_list_2025.csv")
product_url_list=list(product_url_list["product url"])

In [ ]:
def one_page_crawler(input_URL):
  headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}
  #res=requests.get(input_URL,headers=headers)
  session = requests.Session()
  time.sleep(random.randint(1,5))
  html = session.get(input_URL, headers=headers).content
  #with urlopen(input_URL) as response:
  #  html = response.read()
  soup = bs(html, 'html.parser')

  name=[item.get_text() for item in soup.find_all("h1",class_="specs-phone-name-title")][0]

  temp_first_list=[]
  for item in soup.find_all("th"):
    temp_first_list.append(item.get_text().strip())

  temp_key_list=[]
  for item in soup.find_all("td",class_="ttl"):
    temp_key_list.append(item.get_text().strip())

  temp_key_list=[x.strip() for x in temp_key_list if x.strip()!=""]

  data={}
  temp_dict={}
  temp_all_list=[]
  for item in soup.select("tr"):
    temp_list=item.get_text().split("\n")
    temp_list=[x.strip() for x in temp_list if x.strip()!=""]
    if "Tests" in temp_list:
      break
    temp_all_list+=temp_list



  index=0
  data={}
  for item in temp_all_list:
    if item in temp_first_list:
      if item=="Features":
        if temp_all_list[index+1] not in temp_key_list:
          temp_second_key=item
          data[temp_first_key][temp_second_key]=[]
          pass
        else:
          temp_first_key=item
          data[temp_first_key]={}
      else:
        temp_first_key=item
        data[temp_first_key]={}
    elif item in temp_key_list:
      temp_second_key=item
      data[temp_first_key][temp_second_key]=[]

    else:
      try:
        data[temp_first_key][temp_second_key].append(item)
      except:
        data[temp_first_key]=item
    index+=1

  data["Demand"]={}
  data["Demand"]["Fans"]=soup.find_all('strong',class_='accent')[1].get_text()
  data["Demand"]["Rate"]=soup.find_all('li',class_='light pattern help help-popularity')[0].select('strong')[0].get_text().strip()
  data["Demand"]["Hits"]=soup.find_all('li',class_='light pattern help help-popularity')[0].select('span')[0].get_text()

  return name, data

In [ ]:
temp_URL='https://www.gsmarena.com/samsung_x610-800.php'
name, data = one_page_crawler(temp_URL)

In [ ]:
data["Demand"]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Total_data=OrderedDict()
temp_df=pd.DataFrame()
error_list=[]
temp_result_list = []
for temp_URL in tqdm(product_url_list):
  try:
    temp_name, temp_data=one_page_crawler(temp_URL)
    Total_data[temp_name]=temp_data
    temp_list=[]
    temp_list.append(temp_name)
    temp_list.append(temp_data)
    temp_result_list.append(temp_list)
  except:
    print(temp_URL)
    error_list.append(temp_URL)
    pass

In [ ]:
temp_df=pd.DataFrame(temp_result_list)
temp_df.columns=["name","data"]
temp_df.to_csv(data_directory+"/temp_df"+str(len(temp_df))+".csv")
with open(data_directory+'/temp'+str(len(temp_df))+'_utf_8.json','w',encoding="utf-8") as f:
  json.dump(Total_data,f,indent="\t")
with open(data_directory+'/temp'+str(len(temp_df))+'_utg_sig_8.json','w',encoding="utf-8-sig") as f:
  json.dump(Total_data,f,indent="\t")
with open(data_directory+'/temp'+str(len(temp_df))+'_raw.json','w') as f:
  json.dump(Total_data,f,indent="\t")

In [ ]:
import json
with open(data_directory+'/Total_data.json','w') as json_file:
    json.dump(Total_data, json_file)

In [ ]:
import json
with open('/content/drive/MyDrive/Research/Product Science/Data/Total_Data.json','r') as json_file:
    Total_data = json.load(json_file)

In [ ]:
len(Total_data)

In [ ]:
temp_keys = list(Total_data.keys())
temp_keys[0:10]